## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
!python -V

Python 3.9.21


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* **19**

In [6]:
df = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")
df.shape

(3066766, 19)

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* **42.59**
* 52.59
* 62.59

In [7]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [8]:
df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df["duration"].std()

42.594351241920904

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* **98%**

In [9]:
initial_raws = df.shape[0]
initial_raws

3066766

In [10]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
cleaned_raws = df.shape[0]
print(f"After cleaning, it remains {cleaned_raws} that means {(100 * cleaned_raws / initial_raws):.0f} %")

After cleaning, it remains 3009173 that means 98 %


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* **515**
* 715

In [11]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

df[categorical] = df[categorical].astype(str)

In [12]:
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(f"There're {X_train.shape[1]} columns in the matrix")

There're 515 columns in the matrix


## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* **7.64**
* 11.64
* 16.64

In [13]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.64926195318552

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* **7.81**
* 11.81
* 16.81

In [14]:
df_val = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")
df_val["duration"] = df_val["tpep_dropoff_datetime"] - df_val["tpep_pickup_datetime"]
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

In [15]:
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [16]:
val_dicts = df_val[categorical].astype(str).to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
y_val = df_val[target].values

In [18]:
y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811817350047379

## Submit the results

* Submit your results here: https://courses.datatalks.club/mlops-zoomcamp-2025/homework/hw1
* If your answer doesn't match options exactly, select the closest one